In [42]:
import numpy as np
import cv2
import depthai as dai
from pathlib import Path
import time
import numpy as np
import glob
import re

In [39]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')


### Identifying faces in live video feed

and storing the face detected frames in "output_frames"

In [40]:
# using live video 
# cap = cv2.VideoCapture(0)

# using a pre captured video as an example.
cap = cv2.VideoCapture("video.mp4")

# Make sure the destination path is present before starting to store the examples
Path(f"./output_frames/").mkdir(parents=True, exist_ok=True)


while True:
    ret, img = cap.read()
    if not ret:
        print('No frames left!')
        break
        
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    for (x,y,w,h) in faces:
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    cv2.imshow('img',img)
    if cv2.waitKey(10) == ord("q"):
        break
    cv2.imwrite(f"output_frames/{int(time.time() * 10000)}.png", img)
        
cv2.destroyAllWindows()

No frames left!


### Converting the frames back to a video

In [44]:
img_array = []

num = re.compile(r'(\d+)')

def numSort(val):
    parts = num.split(val)
    parts[1::2] = map(int, parts[1::2])
    return parts

for filename in sorted(glob.glob('output_frames/*.png'), key=numSort):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('output_video.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 30, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
